In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
import lime
import lime.lime_tabular
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dataset_folder = 'Datasets/drugscom/'
train_raw = pd.read_csv(str(dataset_folder+'drugsComTrain_raw.csv'))
test_raw = pd.read_csv(str(dataset_folder+'drugsComTest_raw.csv'))

In [3]:
train = train_raw[['review','rating']]
test = train_raw[['review','rating']]
df_all = pd.concat([train,test]).reset_index()
del df_all['index']

In [4]:
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.model_selection import train_test_split

In [5]:
stops = set(stopwords.words('english'))
not_stop = ["aren't","couldn't","didn't","doesn't","don't","hadn't","hasn't","haven't","isn't","mightn't","mustn't","needn't","no","nor","not","shan't","shouldn't","wasn't","weren't","wouldn't"]
for i in not_stop:
    if i in stops:
        stops.remove(i)
    
stemmer = SnowballStemmer('english')
def review_to_words(raw_review):
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    words = letters_only.lower().split()
    meaningful_words = [w for w in words if not w in stops]
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    return( ' '.join(stemming_words))

In [6]:
%time df_all['review_clean'] = df_all['review'].apply(review_to_words)

Wall time: 6min 49s


In [7]:
df_all['sentiment'] = df_all["rating"].apply(lambda x: 1 if x > 5 else 0)

In [8]:
df_train, df_test = train_test_split(df_all, test_size=0.33, random_state=42) 

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None,
                             preprocessor = None, 
                             stop_words = None, 
                             min_df = 2,
                             max_features = 5000
                            )

In [10]:
%time X_train = vectorizer.fit_transform(df_train['review_clean'])
%time X_test = vectorizer.transform(df_test['review_clean'])

Wall time: 11.2 s
Wall time: 5.01 s


In [11]:
y_train = df_train['sentiment']
y_test = df_test['sentiment']

In [12]:
import pickle
pickling_on = open("textrf.pickle","rb")
c = pickle.load(pickling_on)
pickling_on.close()

In [18]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=3, random_state=1)
c.fit(X_train, y_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(y_train, predict_fn(X_train)))
print('Test', sklearn.metrics.accuracy_score(y_test, predict_fn(X_test)))

Train 0.9998704525370483
Test 0.968156156945997


In [13]:
class_names = ['negative','positive']

In [14]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
pip_line = make_pipeline(vectorizer, c)

In [15]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [34]:
df_test.review_clean.iloc[0]

'lyrica almost year first start realli skinni so weight gain didn bother but gain lbs not good go go back gabapentin help get morphin eye dri blurri fart time fat begin year hurt but side affect too much'

In [ ]:
idx = 0
%time exp = explainer.explain_instance(df_test.review_clean.iloc[idx], pip_line.predict_proba, num_features=6, top_labels=1)

In [41]:
exp.as_list(label = 0)

[('didn', 0.13273120519723347),
 ('not', 0.11264088701252714),
 ('gain', 0.10231757664617594),
 ('eye', 0.09774650554997463),
 ('blurri', 0.07033445785436285),
 ('fart', 0.05998285716333729)]

In [16]:
exp_fn = lambda i: explainer.explain_instance(df_test.review_clean.iloc[i], pip_line.predict_proba, num_features=5, top_labels=1)
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in range(len(xtest)):
        exp = exp_fn(i)
        exp1.append(np.array(exp.as_map()[exp.available_labels()[0]]))
    return np.array(exp1)
exp_fn_wrap = lambda x: exp_fn_blk(x, exp_fn)

In [18]:
%time exp1 = exp_fn_blk(df_test.review_clean.iloc[0:10], exp_fn)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Wall time: 11.3 s


In [183]:
exp1 = exp_fn_blk(df_test.review_clean.iloc[0:400], exp_fn)
print('done exp1')
exp2 = exp_fn_blk(df_test.review_clean.iloc[:400], exp_fn)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


done exp1


In [186]:
i = calc_identity(exp1,exp2)
print(i)

(100.0, 0, 400)


In [191]:
s = calc_separability(df_test.review_clean.iloc[:400].values, exp_fn_wrap)
print(s)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


(0, 399, 159201, 0.0)


In [187]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i][j,0])] = exp[i][j,1]
    return enc_exp

In [188]:
exp1.shape

(400,)

In [189]:
enc1 = enc_exp(exp1, 5000)
enc2 = enc_exp(exp2, 5000)

In [190]:
sb = calc_stability(enc1, df_test.sentiment.iloc[:400].values)
print(sb)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


(106, 400)


In [195]:
import pickle
pickling_on = open("textrf.pickle","wb")
pickle.dump(c, pickling_on)
pickling_on.close()